In [1]:
import numpy as np 
import pandas as pd
from scipy.interpolate import CubicSpline

In [61]:
##############   1. prepare raw CDS data     #################

COUNTRY_NUM = 15
OUTPUT_REPO = "EXT_data"

for i in ['1', '3', '5']:
    CDS1Y = pd.read_excel("raw/bbg_cds_data.xlsx", sheet_name=f"CDS{i}Y_values", dtype=str, na_values=['-1'], parse_dates=[0], index_col=0)
    # keep non-NA columns
    CDS_columns = [str(col) for col in CDS1Y.columns if "0" not in str(col)]
    CDS1Y = CDS1Y[CDS_columns[:COUNTRY_NUM]].astype(float)
    CDS1Y.to_csv(f"inputs/{OUTPUT_REPO}/CDS{i}Y.csv")
    # CDS1Y.loc['2003-02-01':].iloc[:85].to_csv(f"inputs/{OUTPUT_REPO}/CDS{i}Y.csv")

In [2]:
##############   2. construct discount value; boostrap Treasury constant maturity curve       #################
# partial credit to Steven Zheng

PFILE = pd.read_csv("raw/FRB_H15.csv", header=0, skiprows=[1,2,3,4,5], dtype=str, na_values=['ND'], parse_dates=[0])

# rename columns using maturity in weeks
new_cols = ['date', 4, 13, 26, 52, 104, 156, 260, 364, 520, 1040, 1560]
rename_dict = { col: new_cols[i] for i, col in enumerate(PFILE.columns)}
PFILE.rename(columns=rename_dict, inplace=True)
# set index, and set values to numeric, % to unit less
PFILE = PFILE.set_index('date')
PFILE = PFILE.astype({col:np.float for col in PFILE.columns}) / 100
# compute discount value
for col in PFILE.columns:
    PFILE[col] = PFILE[col].apply(func= lambda x: 1. / ( (1 + x) ** (float(col) / 52) ))
PFILE = PFILE.dropna(axis=0)

In [3]:
# interpolate to fill weekly maturities using cubic spline
tenor = PFILE.columns.astype(int)
inter_PFILE_list = list()

for i, row in PFILE.iterrows():
    cs = CubicSpline(x=tenor, y=row.values)
    inter_PFILE_list.append([float(cs(t)) for t in range(1,261)])

inter_PFILE = pd.DataFrame(index=PFILE.index, columns=range(1,261), data=inter_PFILE_list)

In [5]:
# PFILE
inter_PFILE.to_csv("inputs/Test_data/PFILE.csv")